In [ ]:
import csv
import os
import re

from process import read_test,merge,merge_c,trans_txt_to_csv,calculate_er_summary
base_path="/home/user_tp/workspace/code/attack/ModelNet40-C/runs/"
folders_all = ['dgcnn_pct_run_1_no_fps','dgcnn_pct_run_1_no_wfps','dgcnn_pct_run_1_no_wrs','dgcnn_pct_run_1_r_rwup_fps','dgcnn_pct_run_1_r_rwup_wfps','dgcnn_pct_run_1_r_rwup_wrs',
               'dgcnn_pct_run_1_up_or_down_fps','dgcnn_pct_run_1_up_or_down_wfps','dgcnn_pct_run_1_up_or_down_wrs',
               'dgcnn_pointnet2_run_1_no_fps','dgcnn_pointnet2_run_1_no_wfps','dgcnn_pointnet2_run_1_no_wrs','dgcnn_pointnet2_run_1_r_rwup_fps','dgcnn_pointnet2_run_1_r_rwup_wfps',
               'dgcnn_pointnet2_run_1_r_rwup_wrs','dgcnn_pointnet2_run_1_up_or_down_fps','dgcnn_pointnet2_run_1_up_or_down_wfps','dgcnn_pointnet2_run_1_up_or_down_wrs',
               'cutmix_k_pct_run_1','cutmix_r_pct_run_1','mixup_pct_run_1','pgd_pct_run_1','rsmix_pct_run_1',
               'cutmix_k_pointnet2_run_1','cutmix_r_pointnet2_run_1','mixup_pointnet2_run_1','rsmix_pointnet2_run_1']


In [ ]:
from timer import ProfileTimer
from PCT_Pytorch.interpolation import Interpolation
from PCT_Pytorch.sampling import process_point_cloud,process_point_cloud_k,process_point_cloud_ratio,process_point_cloud_upper,process_point_cloud_mix,process_point_cloud_ra,process_point_cloud_ori_score,knn,get_normal_vector
from PCT_Pytorch.perbutation import augment_pointcloud,shuffle_pointcloud,jitter_pointcloud, translate_pointcloud, rotate_pointcloud
import torch
import time
import numpy as np
use_upsample = 'up_or_down_ratio_score_2'
device=torch.device('cuda:2')
xyz = torch.rand(32, 1024, 3).to(device)
# device = points.device
B,N,C=xyz.shape
normal = torch.zeros_like(xyz).to(device)
for i in range(xyz.shape[0]):
    normal[i] = get_normal_vector(xyz[i].unsqueeze(0)).squeeze(0)
times = [] 
timer = ProfileTimer("Loop Timer")
# timer.start()
for i in range(100):
    # timer.reset()  # 重置计时器  
    start_time = time.time()  # 开始计时 
      
    if use_upsample=='up_or_down':
        x = process_point_cloud(xyz, 0.03,normal)
    ###modify for the new method,up_or_kdown
    if use_upsample=='up_or_kdown_0.1':
        x = process_point_cloud_k(xyz, 0.03,normal,0.1,0.75)
    if use_upsample=='up_or_kdown_0.2':
        x = process_point_cloud_k(xyz, 0.03,normal,0.2,0.75)
    if use_upsample=='up_or_kdown_0.3':
        x = process_point_cloud_k(xyz, 0.03,normal,0.3,0.5)
    if use_upsample=='up_or_kdown_0.3_2':
        # print("hello")
        x = process_point_cloud_upper(xyz, 0.03,normal,0.3,0.5)
    ###上采样random_k，下采样knn
    if use_upsample=='up_or_kdown_ratio_2':
        x = process_point_cloud_ratio(xyz, 0.03,normal)
    ###上采样random_k，下采样score
    if use_upsample=='up_or_down_ratio_score_2':
        x = process_point_cloud_mix(xyz,0.03,normal)
    ###上采样random_k，下采样random
    if use_upsample=='up_or_ra_ratio_2':
        x = process_point_cloud_ra(xyz, 0.03,normal)
    ###上采样ori，下采样score
    if use_upsample=='oriup_or_down_ratio_score_2':
        x = process_point_cloud_ori_score(xyz, 0.03,normal)
    elif use_upsample=='r_oup':
        I = Interpolation(0.03)
        d, idx_k = knn(xyz, k=20)
        newx = I.random_k_neighbors_shape_invariant_perturb(xyz,d, idx_k, normal)
        x = torch.cat((xyz, newx), dim=1)
    elif use_upsample=='half_roup':
        if N<1024:
            I = Interpolation(0.03)
            d, idx_k = knn(xyz, k=20)
            newx = I.random_k_neighbors_shape_invariant_perturb(xyz,d ,idx_k, normal)
            num_points_to_select = int(1024-N)
            # sampled_indices = torch.randperm(N)[:num_points_to_select]
            x = torch.cat((xyz, newx[:, :num_points_to_select]), dim=1)
        else:
            x = xyz.contiguous()
    elif use_upsample == 'median_hroup':
        I = Interpolation(0.03)
        distance,idx_k = knn(xyz,k=20)
        # pts = I.random_k_neighbors_shape_invariant_perturb(xyz, None, normal)
        def expand_point_cloud_to_1024(original_points, accumulated_points, normal,distance,idx_k):
            if accumulated_points.shape[1] >= 1024:  
                return accumulated_points[:,:1024,:]
            pts_p = I.random_k_neighbors_shape_invariant_perturb(original_points, distance,idx_k, normal)   
            pts = torch.cat((accumulated_points, pts_p), dim=1) 
            return expand_point_cloud_to_1024(original_points, pts, normal,distance,idx_k)
        x = expand_point_cloud_to_1024(xyz.clone(),xyz.clone(),normal,distance,idx_k)
   
    end_time = time.time()  # 结束计时  
    elapsed_time = end_time - start_time  # 计算耗时  
    print(f'{i+1}:{elapsed_time}')
    times.append(elapsed_time)  # 将耗时添加到时间数组中 
  
mean_time = np.mean(times)  
std_time = np.std(times)  # 注意：这里计算的是标准差，如果需要方差请使用np.var(times)  
  
print(f"Mean time: {mean_time:.6f} seconds")  
print(f"Standard deviation: {std_time:.6f} seconds")  
  
# 保存时间数组到.npy文件  
np.save("loop_times.npy", times)

In [1]:
from PCT_Pytorch.sampling import weighted_random_point_sample, cal_weight,filter_random_point_sample
from pointnet2_ops import pointnet2_utils
import torch
import numpy as np
import time
npoint=512
device=torch.device('cuda:2')
point = torch.rand(32, 1024, 3).to(device)
times = [] 
# timer = ProfileTimer("Loop Timer")
# timer.start()
times_weighted = []  
times_filter = []  
times_furthest = []  

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


#时间

In [10]:
import csv  
import math  # 导入math模块以使用sqrt函数   
numbers = []  
count = 0  
batch=32
phase='train' #train test
model='curvenet' #pointnet2 pct curvenet
me='fps'#fps ffps re_wrs fps512 ffps512 fps_clean ffps_clean
filename=f'/home/user_tp/workspace/code/attack/ModelNet40-C/time/cutmix/cutmix_r/train_pointnet2.csv'
# 尝试打开文件并读取第一行（在这种情况下，整个文件只有一行）  
with open(filename, 'r') as file:  
    reader = csv.reader(file)  
    for row in reader:  
        if row:  # 确保行不为空  
            try:  
                # 将行中的所有元素（即逗号分隔的数字）转换为浮点数并添加到列表中  
                numbers = [float(num) for num in row]  
                break  # 因为整个文件只有一行，所以读取完这一行后就可以退出循环  
            except ValueError:  # 如果转换失败（例如，如果行中包含非数字字符）  
                print(f"警告：文件 {filename} 包含非数字字符。")  
                numbers = []  # 可选：清空列表以表示没有有效数据  
                break  # 退出循环  
  
# 计算统计量  
if numbers:  
    mean = sum(numbers) / len(numbers)  
    variance = sum((num - mean) ** 2 for num in numbers) / len(numbers)  
    standard_deviation = variance ** 0.5  
    sample_variance = sum((num - mean) ** 2 for num in numbers) / (len(numbers) - 1) if len(numbers) > 1 else 0  
    sample_standard_deviation = sample_variance ** 0.5  
    print(f"均值: {mean}, 标准差: {standard_deviation}, 样本标准差: {sample_standard_deviation}")  
else:  
    print("没有足够的数据来计算统计量。")


均值: 0.18184903689793178, 标准差: 0.09545559228308799, 样本标准差: 0.095859211809701


In [42]:
from process import merge_c,trans_txt_to_csv,calculate_er_summary
###cal er
MNC_CORRUPTIONS = ["occlusion", "lidar", "density_inc", "density", "cutout", "uniform", "gaussian", "impulse", "upsampling", "background", "rotation", "shear", "distortion", "distortion_rbf", "distortion_rbf_inv"]  
PCC_CORRUPTIONS = ['scale','jitter','dropout_global','dropout_local','add_global','add_local','rotate']
dataset='mnc'
model_c = 'rpc1'
output_mnc=f"/home/user_tp/workspace/code/attack/ModelNet40-C/checkpoints/mnc/mnc-{model_c}/"
output_pcc=f"/home/user_tp/workspace/code/attack/ModelNet40-C/checkpoints/pcc/pcc-{model_c}/"
# output_oo3dc=f"/home/user_tp/workspace/code/attack/ModelNet40-C/runs/oo3dc/oo3dc-{model_c}/"
# output_omnic=f"/home/user_tp/workspace/code/attack/ModelNet40-C/runs/omnic/omnic-{model_c}/"


os.makedirs(output_mnc, exist_ok=True)
os.makedirs(output_pcc, exist_ok=True)
# os.makedirs(output_oo3dc, exist_ok=True)
# os.makedirs(output_omnic, exist_ok=True)


folders_pct = ['dgcnn_pct_run_1_no_fps','dgcnn_pct_run_1_no_wfps','dgcnn_pct_run_1_no_wrs','dgcnn_pct_run_1_r_rwup_fps','dgcnn_pct_run_1_r_rwup_wfps','dgcnn_pct_run_1_r_rwup_wrs',
               'dgcnn_pct_run_1_up_or_down_fps','dgcnn_pct_run_1_up_or_down_wfps','dgcnn_pct_run_1_up_or_down_wrs']
folders_pn2 = ['dgcnn_pointnet2_run_1_no_fps','dgcnn_pointnet2_run_1_no_wfps','dgcnn_pointnet2_run_1_no_wrs','dgcnn_pointnet2_run_1_r_rwup_fps','dgcnn_pointnet2_run_1_r_rwup_wfps','dgcnn_pointnet2_run_1_r_rwup_wrs',
               'dgcnn_pointnet2_run_1_up_or_down_fps','dgcnn_pointnet2_run_1_up_or_down_wfps','dgcnn_pointnet2_run_1_up_or_down_wrs']

fold_pct = ['dgcnn_curvenet_run_1_up_or_down_ratio_score_2_wrs']
fold_pointnet2 = ['dgcnn_pointnet2_run_1_up_or_down_0.1_wrs','dgcnn_pointnet2_run_1_up_or_down_0.2_wrs','dgcnn_pointnet2_run_1_up_or_kdown_0.1_wrs','dgcnn_pointnet2_run_1_up_or_kdown_0.2_wrs']
fold_aug=['dgcnn_curvenet_run_1_no_fps','cutmix_r_curvenet_run_1_no_fps','cutmix_k_curvenet_run_1_no_fps','mixup_curvenet_run_1_no_fps','rsmix_curvenet_run_1_no_fps','pgd_curvenet_run_1_no_fps']
fold_aug_ck =['dgcnn_pct_run_1','cutmix_r_pct_run_1','cutmix_k_pct_run_1','mixup_pct_run_1','rsmix_pct_run_1','pgd_pct_run_1']
# fold_oo3dc=['dgcnn_pointnet_run_1_no_no','cutmix_r_pointnet_run_1_no_no','cutmix_k_pointnet_run_1_no_no','mixup_pointnet_run_1_no_no','rsmix_pointnet_run_1_no_no','pgd_pointnet_run_1_no_no',
#         'dgcnn_gdanet_run_1_no_no','cutmix_r_gdanet_run_1_no_no','cutmix_k_gdanet_run_1_no_no','mixup_gdanet_run_1_no_no','rsmix_gdanet_run_1_no_no','pgd_gdanet_run_1_no_no'
#         ]
fold_c = ['dgcnn_curvenet_run_1_up_or_down_ratio_score_2_wrs','dgcnn_curvenet_run_1_up_or_down_ratio_score_2_wrs3']
fold_uod = ['dgcnn_pct_run_1_up_or_kdown_0.1_wrs','dgcnn_pct_run_1_up_or_kdown_0.2_wrs','dgcnn_pct_run_1_up_or_kdown_0.3_wrs',
            'dgcnn_pointnet2_run_1_up_or_kdown_0.1_wrs','dgcnn_pointnet2_run_1_up_or_kdown_0.2_wrs','dgcnn_pointnet2_run_1_up_or_kdown_0.3_wrs']
# fold_wfps_k_r = ['dgcnn_pct_run_1_up_or_kdown_0.3_2_wrs','dgcnn_pct_run_1_up_or_kdown_ratio_2_wrs','dgcnn_pct_run_1_up_or_down_ratio_score_2_wrs',
                #  'dgcnn_pointnet2_run_1_up_or_kdown_0.3_2_wrs','dgcnn_pointnet2_run_1_up_or_kdown_ratio_2_wrs','dgcnn_pointnet2_run_1_up_or_down_ratio_score_2_wrs']
fold_abla = ['dgcnn_curvenet_run_1_oriup_or_down_ratio_score_2_wrs','dgcnn_curvenet_run_1_up_or_ra_ratio_2_wrs','dgcnn_curvenet_run_1_up_or_kdown_ratio_2_wrs',
             'dgcnn_pct_run_1_oriup_or_down_ratio_score_2_wrs','dgcnn_pct_run_1_up_or_ra_ratio_2_wrs','dgcnn_pct_run_1_up_or_kdown_ratio_2_wrs']
fold_wfps_k_r_8 = ['dgcnn_pct_run_1_up_or_down_0.1_wrs','dgcnn_pct_run_1_up_or_kdown_0.2_wrs','dgcnn_pct_run_1_up_or_kdown_0.3_wrs','dgcnn_pct_run_1_r_rwup_wrs',
                 'dgcnn_pointnet2_run_1_up_or_down_0.1_wrs','dgcnn_pointnet2_run_1_up_or_kdown_0.2_wrs','dgcnn_pointnet2_run_1_up_or_kdown_0.3_wrs','dgcnn_pointnet2_run_1_r_rwup_wrs']
fold_base = ['dgcnn_dgcnn_run_1','dgcnn_rscnn_run_1','dgcnn_simpleview_run_1']
fold_rebuttal = ['dgcnn_pct_run_1_up_or_down_ratio_score_2_wrs_2','dgcnn_pct_run_1_up_or_down_ratio_score_2_wrs_5','dgcnn_pct_run_1_up_or_down_ratio_score_2_wrs',
                 'dgcnn_pct_run_1_up_or_down_ratio_score_2_wrs_50','dgcnn_pct_run_1_up_or_down_ratio_score_2_wrs_200','dgcnn_pct_run_1_up_or_down_ratio_score_2_wrs_500']
fold_rpc=['dgcnn_rpc_run_1_up_or_down_ratio_score_2_wrs']#,'dgcnn_rpc_run_1_WOLFMix',dgcnn_rpc_run_1_up_or_down_ratio_score_2_wrs]

In [43]:
###为了整理实验数据
###修改上面的model_c,pointcloudc需要修改dataset

# for up in ['up_or_down_0.1','up_or_down_0.2','up_or_kdown_0.1','up_or_kdown_0.2']:
#     for down in ['wrs']:
###测试的时候用的方法
if model_c=='ffps-curvenet':
    fold=fold_pct
elif model_c=='pct':
    fold=folders_pct
elif model_c=='pointnet2_down':
    fold=fold_pointnet2

elif model_c =='base-dg-rs-sv':
    fold=fold_base
elif model_c =='sample-abla':
    fold=fold_abla
elif model_c =='ours-curvenet':
    fold=fold_c
elif model_c=='base-da-curvenet':
    fold = fold_aug
elif model_c=='pct-knn':
    fold = fold_rebuttal
elif model_c=='rpc1':
    fold = fold_rpc
else:
    fold=folders_pn2

##dataset
if dataset == 'pcc':
    outputs = output_pcc
    data=PCC_CORRUPTIONS
elif dataset =='mnc':
    outputs=output_mnc
    data=MNC_CORRUPTIONS
elif dataset =='oo3d':
    outputs=output_oo3dc
    data = PCC_CORRUPTIONS
elif dataset =='omnic':
    outputs=output_omnic
    data = PCC_CORRUPTIONS
base_path = '/home/user_tp/workspace/code/attack/ModelNet40-C/checkpoints'
for up in ['median_hroup']:
# for up in ['no']:
#     for down in ['no']:
    for down in ['ffps_0.95']:
    # for down in ['ffps_0.95']: #'wfps_add_0.9','wfps_add_0.95','wfps']:
        for model in ['_best_test']:
# for up in ['no','r_rwup']:
#     for down in ['fps','wfps','wrs']:
            type = f"{up}_{down}{model}"
            output =outputs+f'{dataset}_{type}/'
            os.makedirs(output, exist_ok=True)
            trans_txt_to_csv(base_path,fold,output,dataset,type)
            calculate_er_summary(fold,output,data,type)

input_file_path: /home/user_tp/workspace/code/attack/ModelNet40-C/checkpoints/dgcnn_rpc_run_1_up_or_down_ratio_score_2_wrs/mnc_median_hroup_ffps_0.95_best_test.txt
output_file: /home/user_tp/workspace/code/attack/ModelNet40-C/checkpoints/mnc/mnc-rpc1/mnc_median_hroup_ffps_0.95_best_test/dgcnn_rpc_run_1_up_or_down_ratio_score_2_wrs_median_hroup_ffps_0.95_best_test.csv


In [ ]:
###修改上面的model_c,pointcloudc需要修改dataset
csv_files = []
###
# for up in ['half_roup1024','half_roup']:
    # for down in ['wfps_add','wfps']:
for up in ['median_hroup']:
# for up in ['no']:
    # for down in ['no']:
    # for down in ['ffps_0.95']:
    # for down in ['f_0.95fpsf_0.7','f_0.95fpsf_0.75','f_0.95fpsf_0.8','f_0.95fpsf_0.95','ffps_0.95']:
    for down in ['ffps_0.5','ffps_0.55','ffps_0.6','ffps_0.65','ffps_0.7','ffps_0.75','ffps_0.8','ffps_0.85','ffps_0.9','ffps_0.95','fps']:#'wfps_add_0.9','wfps_add_0.95','wfps']:
        for model in ['_final']:

            type = f"{up}_{down}{model}"
            csv_files.append(f"{dataset}_{type}/er_{type}.csv")
print(csv_files)
merge_c(csv_files,outputs,"file_name","er")

In [ ]:
from process import read_test,merge

In [ ]:

###merge test
output_test =f"/home/user_tp/workspace/code/attack/ModelNet40-C/runs/"
model = 'pct'
folder=[]
output = output_test+f"test-{model}-down/"
os.makedirs(output, exist_ok=True)

for up in ['selfmix_randomts']:
    for down in ['wrs']:
        type = f"{up}_{down}"
        folder.append(f"dgcnn_{model}_run_1_{type}")
print("fold:",folder)
test_files=[]
for up in ['no','r_rwup']:
    for down in ['wfps']:
        type = f"{up}_{down}"
        test_files.append(f"test-{model}-down/{type}.csv")
        read_test(output_test,output,folder,type,model)

# print("test_files:",test_files)
# merge(test_files,output_test,"folder","acc",model)

In [ ]:
import os  
  
def get_first_level_folders(directory):  
    folders = []  
    for item in os.listdir(directory):  
        item_path = os.path.join(directory, item)  
        if os.path.isdir(item_path):  
            folders.append(item)  
    return folders  

In [ ]:
# 使用示例  
folder_path = '/home/user_tp/workspace/code/attack/ModelNet40-C/runs/mnc'  # 请将此路径替换为您要提取文件名的文件夹路径  
file_names_list = get_first_level_folders(folder_path)  
print(file_names_list)

In [ ]:
import pandas as pd    
import matplotlib.pyplot as plt    
from matplotlib.ticker import StrMethodFormatter  

m='pcc'
# CSV文件路径    
file1_path = f'/home/user_tp/workspace/code/attack/ModelNet40-C/runs/{m}/{m}-ffps-pct/er.csv'    
file2_path = f'/home/user_tp/workspace/code/attack/ModelNet40-C/runs/{m}/{m}-ffps-curvenet/er.csv'    
    
# 横轴数据    
x_axis = [0.50, 0.55, 0.60, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90,0.95, 1.00]    
    
def read_csv_data(file_path):    
    # 读取CSV文件，跳过第一行（标题行），并假设第二行是我们需要的数据  
    data = pd.read_csv(file_path, header=None).iloc[1, 1:]   
    num_list = [round(float(item), 4) * 100 for item in data.astype(str)]  # 注意：这里假设data是Series  
    return num_list    
    
# 读取两个CSV文件的数据    
data1 = read_csv_data(file1_path)    
data2 = read_csv_data(file2_path)    
    
# 绘制折线图    
plt.figure(figsize=(8, 6))    
plt.plot(x_axis, data1, label='PCT', color='red', marker='o')    
plt.plot(x_axis, data2, label='CurveNet', color='blue', marker='o')    
    
# 添加图例、标题和坐标轴标签      
# plt.title('Difference proportions of FFPS', fontsize=24)      
plt.xlabel('Threshold $\omega$', fontsize=24)      
plt.ylabel('mER(%)', fontsize=24)      
plt.legend(fontsize=28)    
  
# 设置Y轴刻度标签格式化为一位小数  
plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:.1f}'))  
plt.tick_params(axis='both', which='major', labelsize=16)  # 同时调整x轴和y轴的主要刻度标签字体为16  
# plt.yticks([15.0, 17.5, 20.0, 22.5, 25.0], ['15.0', '17.5', '20.0', '22.5', '25.0'])#mnc
plt.yticks([12.5, 18.5, 24.5, 30.5, 36.5], ['12.5', '18.5', '24.5', '30.5', '36.5'])#pcc
# 设置网格线样式  
plt.grid(True, linestyle='--', linewidth=0.5)     

# 保存图表  
plt.savefig(f'/home/user_tp/workspace/code/attack/ModelNet40-C/visualize/ffps/{m}-ffps.png')#,format='eps',dpi=300)  
  
# 显示图表（如果你是在一个支持图形界面的环境中运行这段代码）  
plt.show()

In [19]:
import os  
import h5py  

file_path = "/home/user_tp/workspace/data/shapenet_c/clean.h5"  

if os.path.exists(file_path):  
    with h5py.File(file_path, 'r') as f:  
        # 获取数据集的名称  
        dataset_name = list(f.keys())[0]  
        
        # 获取数据集的 shape  
        dataset = f[dataset_name]  
        data_shape = dataset.shape  
        
        print(f"{data_shape}") 
        print("data[0]:",dataset[0].shape) 
else:  
    print("File not found.")  

(2874, 2048, 3)
data[0]: (2048, 3)
